In [1]:
import json as j
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2

from nltk.stem import WordNetLemmatizer

In [2]:
data = pd.read_json('resp', orient='coloumns')
data = pd.read_json((data['questions']).to_json(), orient='index')
data.shape

(2000, 12)

In [3]:
remove = ['created_at','creator_id','id','private','question_url','status','updated_at','view_count','category','topic','question_title']
data = data[data.columns.difference(remove)]
data =  data.apply(lambda x: x.astype(str).str.lower())
data

,question_description
0,i loss my hair... how to growth a hair...
1,what is diet recommendations for healthy bones
10,precautions to avoid type 2 diabetes?
100,getting pre masturbation problem how to cure it?
1000,मेरे सर के बाल ञपने आप झड़ जाता है। इसे कैसे रो...
1001,how to gain weight if iam 17 year old
1002,how to solve this problem for headaches
1003,my age is 19years but height is 5'3 so how is ...
1004,panderm plus cream is harmful for face?
1005,my weight is 50 should i loose it how to loose...


In [30]:
import requests
for i in range(340,345):
    string = data['question_description'][i]
    token = '4a2e0c8d189fb8fabf18c339c5436cf6'
    response = requests.get('https://matrix.medikoe.com/api/spellCheck?token='+token+'&test_string='+string, data =string)
    #print(response.text)
    med = response.json()['output']
    print(med)

i want to be rest ,but not possible in busy life , how is this possible

how to gain weight fast within 15 days?
how decrease my weight. ..my weight...is
how do i gain weight...my weight is 48 and i want to increase it.
how to decrease weight by doing good exercise?


In [34]:
import nltk
from nltk.corpus import stopwords
stopwords = nltk.corpus.stopwords.words('english')
corpus =[]
for i in range(0,1200):   
    review = re.sub('[^a-zA-Z]',' ',data['question_description'][i])
    porter = PorterStemmer()
    lemma = WordNetLemmatizer()
    
    string = review
    token = '4a2e0c8d189fb8fabf18c339c5436cf6'
    response = requests.get('https://matrix.medikoe.com/api/spellCheck?token='+token+'&test_string='+string, data =string)
    #print(response.text)
    review = response.json()['output']
    #print(med)
    review = review.split()
    review = [porter.stem(t) for t in review]
    review = [lemma.lemmatize(t) for t in review]
    review = [w for w in review if not w in stopwords]
    review =' '.join(review)
    corpus.append(review)
    
    print(review)

lost hair growth hair
diet recommend healthi bone
decreas regular stress
lot unwant hair hand lag want remov
loss appetit side effect antidiabet probabl solut thi
diet plan type diabet men
ani treatment regain lost speech attack paralysi
reason yellow teeth care taken get back white teeth
precaut avoid type diabet
manag acn natur way
precaut avoid type diabet
possibl solut dri eye
toothach day get rid pain
diet increas weight


KeyboardInterrupt: 

In [35]:
for i in range(0,1200):   
    review = re.sub('[^a-zA-Z]',' ',data['question_description'][i])
    print(review)

i loss my hair    how to growth a hair    
what is diet recommendations for healthy bones
how to decrease the regular stress
i have lot s of unwanted hair on my hand and lags  i want to remove it
is loss of appetite is side effect of antidiabetics     what should be probable solution on this  
what should be diet plan for type ll diabetes in men  
is there any treatment for regaining of lost speech  after an attack of paralysis  
what are the reasons for yellow teeth and care to be taken for getting back white teeth  
precautions to avoid type   diabetes 
how to manage acne the natural way 
precautions to avoid type   diabetes 
what is the possible solution for dry eyes 
i have toothache from few days  how to get rid of pain  
what is a diet for increasing weight  
i have severe back pain  how to get rid of back pain  
how to lower the intensity of migraine headache 
how to gain weight naturally  
how to get rid from cinus problem 
what is the best way to relieve boredom and stress aft

sir   what is the cause of white spots on my face and how to remove them 
suggest me some tips for avoid acne 
                                  
mera wajan bahut kam hai kya karu
it is necessary to donate blood
which medicine to take in fever
what should be eaten while diet
how to stop bleeding from ears
sir how can i increase my health i am thin from childhood 
how can increase platletus in blood
to gain weight what i want to do 
which is the healthy food to gain weight
how i can make my penis straight  
my lips is always hard whats caused sir 
what is the morning energy food
what should i do to keep healthy 
how can i reduce my weight    what are steps i should follow   
what should i do for my health 
how do i loose my weight  i am    kgs  
best food to control blood pressure 
i have a small hump like thing on my back of neck   i also have some back pain problems also  whenever i go for walking my sholders pain alot  what is these symptoms    what to do   
can i get fair  please su

In [49]:
y = pd.read_csv("category.csv")

In [50]:
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.333)

In [73]:
pipeline = Pipeline([('vect', TfidfVectorizer(ngram_range=(0, 1), stop_words="english", sublinear_tf=True)),
                     ('chi',  SelectKBest(chi2, k=900)),
                     ('clf', SVC(kernel ='linear'))])


model = pipeline.fit(X_train, y_train)

C:\Users\Harsh Gupta\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [74]:
vectorizer = model.named_steps['vect']
chi = model.named_steps['chi']
clf = model.named_steps['clf']

In [75]:
feature_names = vectorizer.get_feature_names()
feature_names = [feature_names[i] for i in chi.get_support(indices=True)]
feature_names = np.asarray(feature_names)

In [76]:
feature_names

array(['aan', 'aankho', 'aati', 'aawaj', 'ab', 'abdomen', 'abl', 'accid',
       'accupuntur', 'ach', 'achha', 'achiev', 'achn', 'acid', 'acn',
       'action', 'activ', 'adhik', 'advantag', 'advic', 'affecr', 'affect',
       'age', 'ago', 'alcer', 'alcohol', 'alitha', 'allergi', 'alot',
       'alov', 'alway', 'amentia', 'ana', 'anelu', 'ankel', 'ankl',
       'answer', 'anti', 'antibiot', 'antidiabet', 'antioxid', 'anxieti',
       'anyon', 'anyth', 'appetit', 'appl', 'arm', 'arogyavannu',
       'arthriti', 'articl', 'ask', 'astama', 'asthama', 'avoid', 'awar',
       'away', 'ayurveda', 'azi', 'baad', 'babi', 'backpain', 'bacteria',
       'bad', 'bada', 'bahut', 'balanc', 'banana', 'bar', 'barhega',
       'batawo', 'batey', 'beard', 'beauti', 'becam', 'becom', 'bed',
       'beer', 'beetroot', 'befor', 'belli', 'benefit', 'benifit', 'best',
       'better', 'bhot', 'bicep', 'big', 'bigger', 'biggest', 'bilirubin',
       'black', 'bledington', 'bleed', 'blockapot', 'blood', 'blo

In [77]:
print("accuracy score: " + str(model.score(X_test, y_test)))

accuracy score: 0.75


In [86]:
y_pred = model.predict(X_test)
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test,y_pred)
acc

0.75

In [84]:
print(model.predict(['how to control my powers']))

['Others']


In [80]:
from sklearn.model_selection import cross_val_score
accuracy = cross_val_score(estimator = model, X = X_train, y = y_train, cv=10)
accuracy

C:\Users\Harsh Gupta\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
C:\Users\Harsh Gupta\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Harsh Gupta\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Harsh Gupta\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please ch

array([ 0.74157303,  0.75      ,  0.71084337,  0.63414634,  0.67073171,
        0.71604938,  0.61538462,  0.83116883,  0.7260274 ,  0.77464789])